### Mount google drive

In [ ]:
from google.colab import drive
import sys

drive.mount('/content/drive')
FOLDERNAME = 'CUHK/AIST2010'
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/CUHK/AIST2010


### feature extraction

In [37]:
def min_max_normalize(data):
    # Extract x and y values
    x_values = [item['x'] for item in data]
    y_values = [item['y'] for item in data]

    # Calculate min and max for x and y
    x_min, x_max = min(x_values), max(x_values)
    y_min, y_max = min(y_values), max(y_values)

    # Normalize each dictionary
    normalized_data = []
    for item in data:
        normalized_item = {
            'time': item['time'],
            'x': round((item['x'] - x_min) / (x_max - x_min), 2),
            'y': round((item['y'] - y_min) / (y_max - y_min), 2),
            'z': item['z'],
            'points': item['points'],
        }
        normalized_data.append(normalized_item)

    return normalized_data

22050

In [ ]:
import json

# Load JSON from a file
def load_json_as_dict(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

# Example usage
file_path = 'songData.json'  # Replace with your JSON file path
data_dict = load_json_as_dict(file_path)

In [41]:
import librosa
import numpy as np
import requests
import tempfile

# URL of the audio file
audio_url = ['https://ia801509.us.archive.org/24/items/fcbqy/Abba%20-%20Dancing%20Queen.mp3']

for url in audio_url:
    # Fetch the audio file from the URL
    response = requests.get(url)

    # Use a temporary file to save the audio data
    with tempfile.NamedTemporaryFile(delete=True) as temp_file:
        temp_file.write(response.content)
        temp_file.flush()  # Ensure the data is written

        # Load audio file using librosa
        y, sr = librosa.load(temp_file.name)

        D = np.abs(librosa.stft(y))  # Short-time Fourier transform
        DB = librosa.amplitude_to_db(np.abs(D), ref=np.max)

        # Extract beats
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr)

        # Prepare the beatMap
        beat_map = []
        for beat in beats:
            # Convert frame to time
            time = librosa.frames_to_time(beat, sr=sr)

            # Create the beatMap entry
            beat_map.append({
                'time': round(time,2),
                'x': np.where(DB.T[beat] == max(DB.T[beat]))[0][0],
                'y': round(abs(max(DB.T[beat]))),
                'z': 0,
                'points': 100  # You can adjust this as needed
            })

    # Print the beatMap
    # print(beat_map)

    normalized_data = min_max_normalize(beat_map)
    print(normalized_data)

    for song in data_dict['songs']:
        if song['audioUrl'] == url:
            song['beatMap'] = normalized_data

print("Data extraction complete!")


[{'time': 1.72, 'x': 0.88, 'y': 0.31, 'z': 0, 'points': 100}, {'time': 2.37, 'x': 0.0, 'y': 0.17, 'z': 0, 'points': 100}, {'time': 3.0, 'x': 0.49, 'y': 0.19, 'z': 0, 'points': 100}, {'time': 3.58, 'x': 0.0, 'y': 0.14, 'z': 0, 'points': 100}, {'time': 4.2, 'x': 0.06, 'y': 0.26, 'z': 0, 'points': 100}, {'time': 4.81, 'x': 0.0, 'y': 0.14, 'z': 0, 'points': 100}, {'time': 5.41, 'x': 0.24, 'y': 0.24, 'z': 0, 'points': 100}, {'time': 6.01, 'x': 0.0, 'y': 0.14, 'z': 0, 'points': 100}, {'time': 6.59, 'x': 0.0, 'y': 0.19, 'z': 0, 'points': 100}, {'time': 7.2, 'x': 0.0, 'y': 0.12, 'z': 0, 'points': 100}, {'time': 7.78, 'x': 0.06, 'y': 0.24, 'z': 0, 'points': 100}, {'time': 8.38, 'x': 0.0, 'y': 0.14, 'z': 0, 'points': 100}, {'time': 8.99, 'x': 0.8, 'y': 0.24, 'z': 0, 'points': 100}, {'time': 9.57, 'x': 0.0, 'y': 0.17, 'z': 0, 'points': 100}, {'time': 10.17, 'x': 0.25, 'y': 0.24, 'z': 0, 'points': 100}, {'time': 10.77, 'x': 0.0, 'y': 0.14, 'z': 0, 'points': 100}, {'time': 11.35, 'x': 0.06, 'y': 0.

In [42]:
print(data_dict)  # Now you can use the dictionary

# Step 2: Save the dictionary as a JSON file
with open('songData.json', 'w') as json_file:
    json.dump(data_dict, json_file)

# Optionally, you can print a confirmation message
print("Dictionary saved as data.json")


{'songs': [{'id': 'song1', 'title': 'Dancing Queen', 'category': 'pop', 'difficulty': 'medium', 'audioUrl': 'https://ia801509.us.archive.org/24/items/fcbqy/Abba%20-%20Dancing%20Queen.mp3', 'beatMap': [{'time': 1.72, 'x': 0.88, 'y': 0.31, 'z': 0, 'points': 100}, {'time': 2.37, 'x': 0.0, 'y': 0.17, 'z': 0, 'points': 100}, {'time': 3.0, 'x': 0.49, 'y': 0.19, 'z': 0, 'points': 100}, {'time': 3.58, 'x': 0.0, 'y': 0.14, 'z': 0, 'points': 100}, {'time': 4.2, 'x': 0.06, 'y': 0.26, 'z': 0, 'points': 100}, {'time': 4.81, 'x': 0.0, 'y': 0.14, 'z': 0, 'points': 100}, {'time': 5.41, 'x': 0.24, 'y': 0.24, 'z': 0, 'points': 100}, {'time': 6.01, 'x': 0.0, 'y': 0.14, 'z': 0, 'points': 100}, {'time': 6.59, 'x': 0.0, 'y': 0.19, 'z': 0, 'points': 100}, {'time': 7.2, 'x': 0.0, 'y': 0.12, 'z': 0, 'points': 100}, {'time': 7.78, 'x': 0.06, 'y': 0.24, 'z': 0, 'points': 100}, {'time': 8.38, 'x': 0.0, 'y': 0.14, 'z': 0, 'points': 100}, {'time': 8.99, 'x': 0.8, 'y': 0.24, 'z': 0, 'points': 100}, {'time': 9.57, 'x